In [69]:
# To use the latest publish `kubeflow-metadata` library, you can run:
!pip install --upgrade kubeflow.metadata --user --use-feature=2020-resolver
# Install other packages:
!pip install --upgrade pandas --user --use-feature=2020-resolver
# Then restart the Notebook kernel.

In [70]:
import pip
pip.main(['install','pandas'])
import pandas

pip.main(['install', 'kubeflow.metadata'])
import kubeflow.metadata

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Processing ./.cache/pip/wheels/04/e9/b7/ef010eb3ef0e48343f764fbf1b2eafa6cd967fd468dc9808e5/kubeflow_metadata-0.3.1-py3-none-any.whl
  Using cached ml_metadata-0.21.1-cp36-cp36m-manylinux2010_x86_64.whl (4.9 MB)
  Using cached tensorflow-2.3.0-cp36-cp36m-manylinux2010_x86_64.whl (320.4 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)
  Using cached h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl (2.9 MB)
  Using cached numpy-1.18.5-cp36-cp36m-manylinux1_x86_64.whl (20.1 MB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: keras-preprocessing
    Found existing installation: Keras-Preprocessing 1.1.0
 

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.0 requires six>=1.12.0, but you'll have six 1.11.0 which is incompatible.


In [71]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4

In [72]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

In [73]:
ws1 = metadata.Workspace(
    # Connect to metadata service in namespace kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="workspace_1",
    description="a workspace for testing",
    labels={"n1": "v1"})

MetadataStore with gRPC connection initialized


In [74]:
r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws_1",
)

In [75]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="execution example",
)
print("An execution was created with id %s" % exec.id)

An execution was created with id 27


In [76]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version=date_set_version,
            query="SELECT * FROM mytable"))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

Data set id is 9 with version 'data_set_version_68105a76-35be-4052-9c61-e636c17977a7'


In [77]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))

kubeflow.metadata.metadata.Model(workspace=None, name='MNIST', description='model to recognize handwritten digits', owner='someone@kubeflow.org', uri='gcs://my-bucket/mnist', version='model_version_9240870e-52ce-4c62-8659-07b82fbd5162', model_type='neural network', training_framework={'name': 'tensorflow', 'version': 'v1.0'}, hyperparameters={'learning_rate': 0.5, 'layers': [10, 3, 1], 'early_stop': True}, labels={'mylabel': 'l1'}, id=10, create_time='2020-09-04T03:42:32.469238Z', kwargs={})

Model id is 10 and version is model_version_9240870e-52ce-4c62-8659-07b82fbd5162


In [78]:
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
print("Metrics id is %s" % metrics.id)

Metrics id is 11


In [79]:
serving_application = metadata.Execution(
    name="serving model",
    workspace=ws1,
    description="an execution to represent model serving component",
)
# Noticed we use model name, version, uri to uniquely identify existing model.
served_model = metadata.Model(
    name="MNIST",
    uri="gcs://my-bucket/mnist",
    version=model.version,
)
m=serving_application.log_input(served_model)
print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

Found the mode with id 10 and version 'model_version_9240870e-52ce-4c62-8659-07b82fbd5162'.


In [80]:
pandas.DataFrame.from_dict(ws1.list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,version,owner,model_type,name,description,create_time,uri,training_framework,hyperparameters,labels,kwargs
0,10,workspace_1,run-2020-09-04T03:41:34.877645,model_version_9240870e-52ce-4c62-8659-07b82fbd...,someone@kubeflow.org,neural network,MNIST,model to recognize handwritten digits,2020-09-04T03:42:32.469238Z,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}


In [81]:
print("Model id is %s\n" % model.id)
    
model_events = ws1.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print("All executions related to the model are {}".format(execution_ids))
# assert execution_ids == set([serving_application.id, exec.id])

trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])

Model id is 10

All executions related to the model are {27, 28}
All artifacts related to the training event are {9, 10, 11}
